# wordslab-notebooks-lib.tools

> Predefined tools for LLMs

In [ ]:
#| default_exp tools

In [ ]:
#| hide
from nbdev.showdoc import *

## Solveit exploration

Here is the list of unique Answerai library names sorted in alphabetical order:

### Librairies providing tools

| Library | GitHub Repo URL | Library Description |
|---------|-----------------|---------------------|
| **contextkit** | [*](https://github.com/AnswerDotAI/ContextKit "GitHub - AnswerDotAI/ContextKit: Useful LLM contexts ready to be used in AIMagic") https://github.com/AnswerDotAI/ContextKit | [*](https://github.com/AnswerDotAI/ContextKit "GitHub - AnswerDotAI/ContextKit: Useful LLM contexts ready to be used in AIMagic") Useful LLM contexts ready to be used in AIMagic. Library for gathering context from various sources (URLs, GitHub repos, Google Docs, PDFs, arXiv) for feeding to LLMs. |
| **dialoghelper** | [*](https://github.com/AnswerDotAI/dialoghelper "GitHub - AnswerDotAI/dialoghelper: Helper functions for solveit dialogs") https://github.com/AnswerDotAI/dialoghelper | [*](https://github.com/AnswerDotAI/dialoghelper "GitHub - AnswerDotAI/dialoghelper: Helper functions for solveit dialogs") Helper functions for solveit dialogs. Provides functions for message manipulation, gist management, screen capture, and tool integration in the solveit environment. |
| **execnb** | [*](https://github.com/AnswerDotAI/execnb "GitHub - AnswerDotAI/execnb: Execute a jupyter notebook, fast, without needing jupyter") https://github.com/AnswerDotAI/execnb | [*](https://github.com/AnswerDotAI/execnb "GitHub - AnswerDotAI/execnb: Execute a jupyter notebook, fast, without needing jupyter") Execute a jupyter notebook, fast, without needing jupyter. Provides CaptureShell for running code and capturing outputs without a Jupyter server. |
| **fastcore** | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") https://github.com/AnswerDotAI/fastcore | Python utilities and enhancements used across fast.ai projects. Includes foundational tools like `@patch`, `@delegates`, type dispatch, and LLM file/bash tools. |
| **ipykernel_helper** | PRIVATE REPO | Helper utilities for IPython/Jupyter kernels, providing `read_url`, `transient` display, enhanced completion, and namespace inspection for dialog environments. |
| **llms_txt** | [*](https://github.com/AnswerDotAI/llms-txt/blob/main/nbs/index.qmd "llms-txt/nbs/index.qmd at main · AnswerDotAI/llms-txt") https://github.com/AnswerDotAI/llms-txt | [*](https://github.com/AnswerDotAI/llms-txt/blob/main/nbs/index.qmd "llms-txt/nbs/index.qmd at main · AnswerDotAI/llms-txt") The `llms.txt` specification is open for community input. A GitHub repository hosts this informal overview. Tools for the /llms.txt standard helping language models use websites. |
| **playwrightnb** | https://github.com/AnswerDotAI/playwrightnb | Playwright browser automation integration for Jupyter notebooks, enabling headless browser control and web scraping. |
| **toolslm** | [*](https://github.com/AnswerDotAI/toolslm "GitHub - AnswerDotAI/toolslm: Tools to make language models a bit easier to use") https://github.com/AnswerDotAI/toolslm | [*](https://github.com/AnswerDotAI/toolslm "GitHub - AnswerDotAI/toolslm: Tools to make language models a bit easier to use") Tools to make language models a bit easier to use. Provides XML context creation, function schema generation, shell execution, and web content downloading for LLMs. |

### Librairies using tools

| Library | GitHub Repo URL | Library Description |
|---------|-----------------|---------------------|
| **claudette** | [*](https://github.com/AnswerDotAI/claudette "GitHub - AnswerDotAI/claudette: Claudette is Claude's friend") [*](https://github.com/AnswerDotAI/claudette "GitHub - AnswerDotAI/claudette: Claudette is Claude's friend") [*](https://github.com/AnswerDotAI/claudette "GitHub - AnswerDotAI/claudette: Claudette is Claude's friend") https://github.com/AnswerDotAI/claudette | [*](https://github.com/AnswerDotAI/claudette "GitHub - AnswerDotAI/claudette: Claudette is Claude's friend") Wrapper for Anthropic's Python SDK. The SDK works well, but is quite low level. Claudette automates pretty much everything that can be automated, whilst providing full control. |
| **cosette** | [*](https://github.com/AnswerDotAI/cosette "GitHub - AnswerDotAI/cosette: Claudette's sister, a helper for OpenAI GPT") https://github.com/AnswerDotAI/cosette | [*](https://github.com/AnswerDotAI/cosette "GitHub - AnswerDotAI/cosette: Claudette's sister, a helper for OpenAI GPT") Claudette's sister, a helper for OpenAI GPT. High-level wrapper for OpenAI's SDK with stateful chat, tool calling, and streaming support. |
| **lisette** | https://github.com/AnswerDotAI/lisette | LiteLLM helper providing unified access to 100+ LLM providers using the OpenAI API format, with stateful chat, tools, web search, and reasoning support. |
| **msglm** | https://github.com/AnswerDotAI/msglm | Makes it easier to create messages for LLMs (Claude, OpenAI). Handles text, images, PDFs, prompt caching, and API-specific formatting. |

### Development tools

| Library | GitHub Repo URL | Library Description |
|---------|-----------------|---------------------|
| **fastcaddy** | https://github.com/AnswerDotAI/fastcaddy | Python wrapper for Caddy web server integration, simplifying HTTPS setup and reverse proxy configuration for FastHTML apps. |
| **fastlite** | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") https://github.com/AnswerDotAI/fastlite | A thin, expressive wrapper around SQLite for Python, providing easy database operations with minimal boilerplate. |
| **fastlucide** | https://github.com/AnswerDotAI/fastlucide | Lucide icons integration for FastHTML applications, providing SVG icons as Python components. |
| **fasthtml** | https://github.com/AnswerDotAI/fasthtml | Modern Python web framework for building HTML applications with HTMX and Starlette, using pure Python for HTML generation. |
| **ghapi** | https://github.com/AnswerDotAI/ghapi | [*](https://github.com/orgs/AnswerDotAI/repositories "AnswerDotAI repositories · GitHub") A delightful and complete interface to GitHub's amazing API. Auto-generated Python client covering all GitHub API endpoints. |
| **monsterui** | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") https://github.com/AnswerDotAI/MonsterUI | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") MonsterUI is a FastHTML Tailwind-powered UI framework for building beautiful web interfaces with minimal code. |
| **nbdev** | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") https://github.com/AnswerDotAI/nbdev | Literate programming framework that allows writing library code, tests, and documentation in Jupyter Notebooks, then exporting to Python modules. |
| **shell_sage** | https://github.com/AnswerDotAI/shell_sage | [*](https://github.com/orgs/AnswerDotAI/repositories "AnswerDotAI repositories · GitHub") ShellSage saves sysadmins' sanity by solving shell script snafus super swiftly. AI-powered shell command helper. |

## Tools catalog

Here is the list of tools we want to develop.

### Enhance IPython kernel

#### ipykernel_helper

ipykernel_helper is a library that provides helper utilities for working with IPython/Jupyter kernels. 

It appears to be part of the infrastructure that makes the Dialog Engineering environment work smoothly, providing convenient functions that are useful in an interactive coding environment.

**IPython InteractiveShell - Complete Feature Summary**

`InteractiveShell` is the core class of IPython, providing an enhanced, interactive Python environment. It's a singleton class that manages everything from code execution to history, completion, and output formatting.

1. **Code Execution**

| Method | Description |
|--------|-------------|
| `run_cell(raw_cell, store_history, silent, shell_futures, cell_id)` | Run a complete IPython cell including magics |
| `run_cell_async(...)` | Async version of `run_cell` |
| `run_code(code_obj, result, async_)` | Execute a compiled code object |
| `run_ast_nodes(nodelist, cell_name, interactivity, compiler, result)` | Run a sequence of AST nodes |
| `run_line_magic(magic_name, line)` | Execute a line magic like `%timeit` |
| `run_cell_magic(magic_name, line, cell)` | Execute a cell magic like `%%bash` |
| `safe_execfile(fname, *where, ...)` | Safely execute a .py file |
| `safe_execfile_ipy(fname, ...)` | Execute .ipy or .ipynb files with IPython syntax |
| `safe_run_module(mod_name, where)` | Safe version of `runpy.run_module()` |

2. **Namespace Management**

| Method/Property | Description |
|-----------------|-------------|
| `user_ns` | The user's namespace dictionary |
| `user_ns_hidden` | Hidden namespace items (not shown to user) |
| `all_ns_refs` | List of all namespace dictionaries where objects may be stored |
| `push(variables, interactive)` | Inject variables into user namespace |
| `del_var(varname, by_name)` | Delete a variable from namespaces |
| `drop_by_id(variables)` | Remove variables if they match given values |
| `ev(expr)` | Evaluate Python expression in user namespace |
| `ex(cmd)` | Execute Python statement in user namespace |
| `init_user_ns()` | Initialize user-visible namespaces to defaults |
| `reset(new_session, aggressive)` | Clear all internal namespaces |
| `reset_selective(regex)` | Clear variables matching a regex pattern |

3. **Code Completion**

| Method | Description |
|--------|-------------|
| `init_completer()` | Initialize completion machinery |
| `complete(text, line, cursor_pos)` | Return completed text and list of completions |
| `set_completer_frame(frame)` | Set the frame for completion context |
| `set_custom_completer(completer, pos)` | Add a custom completer function |

4. **Code Transformation**

| Method/Property | Description |
|-----------------|-------------|
| `transform_cell(raw_cell)` | Transform input cell before parsing (handles `%magic`, `!system`, etc.) |
| `transform_ast(node)` | Apply AST transformations from `ast_transformers` |
| `input_transformers_post` | List of string transformers applied after IPython's own |
| `ast_transformers` | List of `ast.NodeTransformer` instances for code modification |
| `check_complete(code)` | Check if code is ready to execute or needs continuation |

5. **History Management**

| Method/Property | Description |
|-----------------|-------------|
| `init_history()` | Setup command history and autosaves |
| `extract_input_lines(range_str, raw)` | Return input history slices as string |
| `history_length` | Total length of command history |
| `history_load_length` | Number of entries loaded at startup |

6. **Magic System**

| Method | Description |
|--------|-------------|
| `find_magic(magic_name, magic_kind)` | Find and return a magic by name |
| `find_line_magic(magic_name)` | Find a line magic |
| `find_cell_magic(magic_name)` | Find a cell magic |
| `register_magic_function(func, magic_kind, magic_name)` | Expose a function as a magic |
| `define_macro(name, themacro)` | Define a new macro |

7. **Object Inspection**

| Method | Description |
|--------|-------------|
| `object_inspect(oname, detail_level)` | Get object info |
| `object_inspect_mime(oname, detail_level, omit_sections)` | Get object info as mimebundle |
| `object_inspect_text(oname, detail_level)` | Get object info as formatted text |
| `find_user_code(target, raw, py_only, ...)` | Get code from history, file, URL, or string |

8. **Error Handling & Debugging**

| Method/Property | Description |
|-----------------|-------------|
| `showtraceback(exc_tuple, filename, tb_offset, ...)` | Display exception that just occurred |
| `showsyntaxerror(filename, running_compiled_code)` | Display syntax error |
| `showindentationerror()` | Handle IndentationError |
| `get_exception_only(exc_tuple)` | Get exception string without traceback |
| `set_custom_exc(exc_tuple, handler)` | Set custom exception handler |
| `excepthook(etype, value, tb)` | Custom sys.excepthook |
| `debugger(force)` | Call pdb debugger |
| `call_pdb` | Control auto-activation of pdb at exceptions |
| `pdb` | Automatically call pdb after every exception |
| `xmode` | Switch modes for exception handlers |

9. **System Interaction**

| Method | Description |
|--------|-------------|
| `system(cmd)` | Call cmd in subprocess, piping stdout/err |
| `system_piped(cmd)` | Call cmd with piped output |
| `system_raw(cmd)` | Call cmd using os.system or subprocess |
| `getoutput(cmd, split, depth)` | Get output from subprocess |

10. **Expression Evaluation**

| Method | Description |
|--------|-------------|
| `user_expressions(expressions)` | Evaluate dict of expressions, return rich mime-typed display_data |
| `var_expand(cmd, depth, formatter)` | Expand Python variables in a string |

11. **Async Support**

| Property/Method | Description |
|-----------------|-------------|
| `autoawait` | Automatically run await in top-level REPL |
| `should_run_async(raw_cell, ...)` | Determine if cell should run via coroutine runner |
| `loop_runner` | Select the loop runner for async code |

12. **Configuration Properties**

| Property | Description |
|----------|-------------|
| `autocall` | Auto-call callable objects (0=off, 1=smart, 2=full) |
| `autoindent` | Auto-indent code |
| `automagic` | Call magics without leading `%` |
| `colors` | Color scheme (nocolor, neutral, linux, lightbg) |
| `cache_size` | Output cache size (default 1000) |
| `ast_node_interactivity` | Which nodes display output ('all', 'last', 'last_expr', 'none', 'last_expr_or_assign') |
| `banner1` / `banner2` | Banner text before/after profile |
| `display_page` | Show pager content as regular output |
| `enable_html_pager` | Enable HTML in pagers |
| `enable_tip` | Show tip on IPython start |
| `show_rewritten_input` | Show rewritten input for autocall |
| `sphinxify_docstring` | Enable rich HTML docstrings |
| `warn_venv` | Warn if running in venv without IPython installed |

**Architecture Summary**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    InteractiveShell (Singleton)                     │
├─────────────────────────────────────────────────────────────────────┤
│  NAMESPACES                                                         │
│    user_ns, user_ns_hidden, all_ns_refs                            │
├─────────────────────────────────────────────────────────────────────┤
│  CODE EXECUTION PIPELINE                                            │
│    raw_cell → transform_cell → parse → transform_ast → run_code    │
├─────────────────────────────────────────────────────────────────────┤
│  SUBSYSTEMS                                                         │
│    • Completer (tab completion)                                     │
│    • History Manager                                                │
│    • Magic System (line/cell magics)                               │
│    • Display Publisher                                              │
│    • Exception Handler / Debugger                                   │
│    • Input Transformers (magics, shell commands)                   │
│    • AST Transformers                                               │
├─────────────────────────────────────────────────────────────────────┤
│  EXTENSIONS                                                         │
│    • Hooks (set_hook)                                               │
│    • Custom completers (set_custom_completer)                      │
│    • Custom exception handlers (set_custom_exc)                    │
│    • Magic functions (register_magic_function)                     │
└─────────────────────────────────────────────────────────────────────┘
```

**How ipykernel_helper Extends InteractiveShell**

The `ipykernel_helper` library uses fastcore's `@patch` to add methods to `InteractiveShell`:

| Added Method | Purpose |
|--------------|---------|
| `user_items()` | Get user-defined vars & functions (filtered) |
| `get_vars(vs)` | Get specific variable values |
| `get_schemas(fs)` | Get JSON schemas for functions (tool calling) |
| `ranked_complete(code, ...)` | Enhanced completion with ranking |
| `sig_help(code, ...)` | Signature help using Jedi |
| `xpush(**kw)` | Push with kwargs |
| `publish(data, ...)` | Enhanced display publishing |

This is how solveit extends IPython's core to support AI-assisted coding!

#### Extract information from the user's namespace

These are methods patched onto IPython's InteractiveShell by ipykernel_helper. 

Getting an InteractiveShell instance from a notebook code cell is straightforward.

```python
from IPython import get_ipython
shell = get_ipython()
```

They help extract information from the user's namespace for use in dialog/AI contexts.

```python
InteractiveShell.user_items(self, max_len=200, xtra_skip=())
```

Purpose: Get user-defined variables and functions from the IPython namespace, filtering out system/internal items.

Returns: Tuple of (user_vars, user_fns)
- user_vars: Dict of {var_name: repr_string} - variable names and their string representations (truncated to max_len)
- user_fns: Dict of {func_name: signature_string} - user-defined function names and their signatures

What it filters out:
- Hidden namespace items (user_ns_hidden)
- Items starting with _
- Types, modules, methods, builtins, typing constructs
- Specific names like nbmeta, receive_nbmeta

Use case: Let the AI know what variables and functions the user has defined in their session.

```python
InteractiveShell.get_vars(self:InteractiveShell, vs:list, literal=True)
```

Purpose: Retrieve specific variable values from the namespace

Parameters:
- vs: List of variable names to retrieve
- literal: If True, try to return actual Python literals; if False, return string representation

Returns: 
- Dict of {var_name: value} for variables that exist in the namespace

The literal=True behavior:
- Tries to repr() the value, then literal_eval() it back
- If that succeeds, returns the actual value (works for dicts, lists, strings, numbers, etc.)
- If it fails (complex objects), returns the string representation instead

Use case: When the user shares specific variables with the AI using `$varname` notation, this function retrieves their current values.

```python
InteractiveShell.get_schemas(self:InteractiveShell, fs:list)
```

Purpose: Get JSON schemas for functions (for LLM tool calling).

Parameters:
- fs: List of function names to get schemas for

Returns: Dict of {func_name: schema_or_error}
- If successful: {'type': 'function', 'function': {JSON schema}}
- If not found: "funcname not found. Did you run it?"
- If schema error: "funcname: error message"

The schema format (via `toolslm.funccall.get_schema`):

```json
{
    'type': 'function',
    'function': {
        'name': 'my_func',
        'description': 'Docstring here',
        'parameters': {
            'type': 'object',
            'properties': {
                'param1': {'type': 'string', 'description': '...'},
                'param2': {'type': 'integer', 'description': '...'}
            },
            'required': ['param1']
        }
    }
}
```

Use case: When the user shares tools with the AI using `&toolname` notation, this function generates the JSON schemas that tell the AI how to call those functions.

#### Solveit use of get_vars() and get_schema()

The `$varname` and `&toolname` Notation System

Overview

When you write a prompt in solveit containing `$myvar` or `&mytool`, the system:
1. **Parses** your prompt for these special notations
2. **Extracts** the referenced names
3. **Retrieves** their values/schemas from the Python namespace
4. **Injects** this information into the context sent to the AI

**Variable Sharing: `$varname`**

What Happens

```
User writes prompt:
┌─────────────────────────────────────────────┐
│ "Analyze the data in `$df` and tell me     │
│  the average of the 'sales' column"         │
└─────────────────────────────────────────────┘
                    │
                    ▼
            Parse for $-prefixed names
                    │
                    ▼
            Found: ['df']
                    │
                    ▼
            Call: shell.get_vars(['df'])
                    │
                    ▼
            Retrieve current value of df
                    │
                    ▼
┌─────────────────────────────────────────────┐
│ Context sent to AI includes:                │
│                                             │
│ <variables>                                 │
│   df = DataFrame with 100 rows, columns:    │
│        ['date', 'sales', 'region', ...]     │
│ </variables>                                │
│                                             │
│ User prompt: "Analyze the data in df..."    │
└─────────────────────────────────────────────┘
```

Key Behavior

1. Current value: The variable's value at the time the prompt is sent (not when it was first defined)
2. Retroactive updates: If you change the variable and re-run, the context updates
3. Safe representation: Uses `_safe_repr()` to truncate large values (default 200 chars)
4. Literal conversion: Tries to preserve actual Python types when possible via `literal_eval`

Example Flow

```python
# Cell 1: Define data
df = pd.DataFrame({'a': [1,2,3], 'b': [4,5,6]})

# Cell 2: Prompt with $df
# "What's the sum of column 'a' in `$df`?"

# Behind the scenes:
shell.get_vars(['df'])
# Returns: {'df': "   a  b\n0  1  4\n1  2  5\n2  3  6"}
```

**Tool Sharing: `&toolname`**

What Happens

```
User writes prompt:
┌─────────────────────────────────────────────┐
│ "Use `&calculate` to add 15 and 27"         │
└─────────────────────────────────────────────┘
                    │
                    ▼
            Parse for &-prefixed names
                    │
                    ▼
            Found: ['calculate']
                    │
                    ▼
            Call: shell.get_schemas(['calculate'])
                    │
                    ▼
            Generate JSON schema from function
                    │
                    ▼
┌─────────────────────────────────────────────┐
│ AI receives tool definition:                │
│                                             │
│ {                                           │
│   "type": "function",                       │
│   "function": {                             │
│     "name": "calculate",                    │
│     "description": "Add two numbers",       │
│     "parameters": {                         │
│       "type": "object",                     │
│       "properties": {                       │
│         "a": {"type": "integer"},           │
│         "b": {"type": "integer"}            │
│       },                                    │
│       "required": ["a", "b"]                │
│     }                                       │
│   }                                         │
│ }                                           │
└─────────────────────────────────────────────┘
```

**Requirements for Tools**

From `dialoghelper.core.is_usable_tool()`, a function must have:
1. Type annotations for all parameters
2. A docstring (becomes the tool description)

```python
# ✅ Valid tool
def calculate(a: int, b: int) -> int:
    "Add two numbers together"
    return a + b

# ❌ Not usable - no docstring
def bad_tool(a: int, b: int) -> int:
    return a + b

# ❌ Not usable - missing type hints
def another_bad(a, b):
    "Add numbers"
    return a + b
```

**Tool Execution Flow**

When the AI decides to use a tool:

```
┌─────────────────────────────────────────────┐
│ AI Response:                                │
│ "I'll use calculate to add those numbers"   │
│                                             │
│ Tool Call:                                  │
│   name: "calculate"                         │
│   arguments: {"a": 15, "b": 27}             │
└─────────────────────────────────────────────┘
                    │
                    ▼
            solveit receives tool call
                    │
                    ▼
            Look up 'calculate' in namespace
                    │
                    ▼
            Execute: calculate(a=15, b=27)
                    │
                    ▼
            Result: 42
                    │
                    ▼
┌─────────────────────────────────────────────┐
│ Tool result sent back to AI                 │
│                                             │
│ AI continues: "The sum of 15 and 27 is 42"  │
└─────────────────────────────────────────────┘
```

**The Parsing Mechanism**

The parsing likely uses regex to find these patterns:

```python
import re

def parse_prompt(prompt):
    # Find $varname in backticks
    var_pattern = r'`\$(\w+)`'
    vars_found = re.findall(var_pattern, prompt)
    
    # Find &toolname in backticks
    tool_pattern = r'`&(\w+)`'
    tools_found = re.findall(tool_pattern, prompt)
    
    return vars_found, tools_found

# Example
prompt = "Use `&calculate` on `$x` and `$y`"
parse_prompt(prompt)
# Returns: (['x', 'y'], ['calculate'])
```

**Complete Architecture**

```
┌─────────────────────────────────────────────────────────────────────┐
│                      USER WRITES PROMPT                             │
│  "Use `&my_tool` to process `$my_data` and return the result"      │
└─────────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      SOLVEIT FRONTEND                               │
│  1. Parse prompt for `$name` and `&name` patterns                  │
│  2. Extract: vars=['my_data'], tools=['my_tool']                   │
│  3. Request from kernel: get_vars(), get_schemas()                 │
└─────────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      IPYKERNEL (Python)                             │
│  shell.get_vars(['my_data'])                                       │
│    → {'my_data': [1, 2, 3, 4, 5]}                                  │
│                                                                     │
│  shell.get_schemas(['my_tool'])                                    │
│    → {'my_tool': {'type': 'function', 'function': {...}}}          │
└─────────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      CONTEXT ASSEMBLY                               │
│  <variables>                                                        │
│    my_data = [1, 2, 3, 4, 5]                                       │
│  </variables>                                                       │
│                                                                     │
│  Tools: [my_tool schema]                                           │
│                                                                     │
│  Prompt: "Use my_tool to process my_data..."                       │
└─────────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      AI (Claude/etc)                                │
│  - Sees variable values in context                                 │
│  - Has tool schemas available                                      │
│  - Can call tools, results executed in user's kernel               │
└─────────────────────────────────────────────────────────────────────┘
```

**Key Points**

| Aspect | `$varname` | `&toolname` |
|--------|-----------|-------------|
| Purpose | Share variable value | Share function as callable tool |
| Retrieval | `get_vars()` | `get_schemas()` |
| What AI sees | Current value (as string/literal) | JSON schema for tool calling |
| Requirements | Variable must exist | Function needs types + docstring |
| AI can modify? | No (read-only snapshot) | Yes (can call the function) |

**Practical Tips**

1. Variables are snapshots: The AI sees the value when you send the prompt
2. Tools execute in your kernel: Side effects happen in your session
3. Use backticks: The notation is `` `$var` `` and `` `&tool` ``, not bare `$var`
4. Check tool validity: Use `is_usable_tool(func)` to verify a function can be used as a tool
5. List tools: Use `mk_toollist(symbols)` from dialoghelper to see available tools formatted for prompts

#### Customize Jupyterlab default behavior to enhance variables display

Let me explain how JupyterLab's Debugger panel works and whether you could hook into it.

**How JupyterLab's Debugger Panel Gets Variables**

The Debugger panel in JupyterLab uses a completely different mechanism than these ipykernel_helper functions:
1. Debug Adapter Protocol (DAP) - JupyterLab's debugger communicates via the DAP standard
2. debugpy - The Python debugger backend 
3. Kernel debug messages - Special debug_request/debug_reply messages over ZMQ
The flow looks like:

```
JupyterLab Debugger UI
        │
        ▼ (DAP protocol)
    debugpy 
        │
        ▼ (inspects frames)
    Python runtime
```
    
**The Challenge**

The user_items(), get_vars(), get_schemas() functions are:
- Pull-based: Called on demand by the solveit frontend
- Custom protocol: Use transient display messages or custom endpoints
- Namespace-focused: Look at InteractiveShell.user_ns

JupyterLab's Debugger is:
- DAP-based: Uses standardized debug adapter protocol
- Frame-focused: Inspects stack frames, not just the namespace
- debugpy-controlled: The variable inspection happens inside debugpy

**Option: Custom Variable Inspector Extension**

Instead of modifying the Debugger, create a JupyterLab extension that:
- Sends custom comm messages to the kernel
- Calls user_items() or similar
- Displays results in a custom panel

```python
# Kernel side
from ipykernel.comm import Comm

def send_variables():
    shell = get_ipython()
    user_vars, user_fns = shell.user_items()
    comm = Comm(target_name='variable_inspector')
    comm.send({'vars': user_vars, 'fns': user_fns})
```

**Practical Recommendation**

The easiest approach that gives you similar functionality to solveit's variable display in standard JupyterLab:

```python
# In your notebook or startup file
from IPython import get_ipython
from IPython.display import display, HTML
import json

def show_user_items(max_len=100):
    """Display user variables and functions like solveit does"""
    shell = get_ipython()
    user_vars, user_fns = shell.user_items(max_len=max_len)
    
    html = "<details open><summary><b>Variables</b></summary><ul>"
    for k, v in user_vars.items():
        html += f"<li><code>{k}</code>: {v}</li>"
    html += "</ul></details>"
    
    html += "<details open><summary><b>Functions</b></summary><ul>"
    for k, v in user_fns.items():
        html += f"<li><code>{k}{v}</code></li>"
    html += "</ul></details>"
    
    display(HTML(html))

# Auto-show after each cell (optional)
# get_ipython().events.register('post_run_cell', lambda r: show_user_items())
```

#### Provide smarter code completion and signature help

Why These Are Better Than Default IPython

Default IPython completion:
- No custom ranking (alphabetical or frequency-based)
- Includes all __dunder__ methods always
- Doesn't prioritize user code

These custom versions:
- Smart ranking: user code > builtins > everything else
- Filters __dunder__ unless explicitly requested
- Context-aware (knows about your namespace)
- Provides structured data for rich UI display

```python
InteractiveShell.ranked_complete(self:InteractiveShell, code, line_no=None, col_no=None)
```

Purpose: Provide intelligent, ranked code completions using Jedi, with custom ranking logic.

Parameters:
- code: The code string to complete
- line_no: Optional line number (1-indexed)
- col_no: Optional column number (1-indexed)

Returns: List of completion objects with these attributes:
- text - The completion text
- type - Type of completion (param, function, module, etc.)
- mod - Module where the item is defined
- rank - Numeric rank (lower = higher priority)

Ranking logic (lower is better):
- Rank 1: Parameters (function arguments being filled)
- Rank 2: Local variables/functions (from __main__)
- Rank 3: Module members (when completing module.something)
- Rank 4: Builtins (like print, len, etc.)
- Rank 5: Everything else (imported modules, third-party)
- Rank +0.1: Private items (starting with _) - slightly lower priority

Special handling:
- Filters out __dunder__ methods unless the user explicitly types __ (so they're not cluttering normal completion)
- Deprioritizes private _methods slightly but doesn't remove them

Use case: Provide better autocomplete suggestions in the solveit dialog environment, prioritizing user-defined items and parameters over stdlib/third-party items.

```python
InteractiveShell.sig_help(self:InteractiveShell, code, line_no=None, col_no=None)
```

Purpose: Get function signature information at the cursor position (like when you type func( and want to see the parameters).

Parameters:
- code	The code string
- line_no	Line number where cursor is (1-indexed)
- col_no	Column number where cursor is (1-indexed)

Returns: List of signature objects, each containing:
- label - Full signature description (e.g., "print(value, ..., sep=' ', end='\n')")
- typ - Type of callable (function, method, class, etc.)
- mod - Module name where it's defined
d- oc - Full docstring
- idx - Current parameter index (which parameter the cursor is on)
- params - List of parameter dicts with name and desc

How it works:
- Uses Jedi's Interpreter with the current namespace to get context-aware signatures
- Falls back to Script (static analysis) if Interpreter doesn't find anything
- Extracts detailed information about each signature
- Returns structured data about parameters and documentation

Use case: Power the signature help tooltip in the solveit editor, showing:
- What parameters a function takes
- Which parameter you're currently typing
- Documentation for each parameter
- Full docstring

#### Override IPython's default behavior to enhance completions

JupyterLab communicates with the kernel via the Jupyter messaging protocol:

```
┌─────────────────┐                    ┌─────────────────┐
│   JupyterLab    │  complete_request  │    IPython      │
│   Frontend      │ ─────────────────► │    Kernel       │
│                 │                    │                 │
│                 │ ◄───────────────── │                 │
│                 │  complete_reply    │                 │
└─────────────────┘                    └─────────────────┘
```

The relevant message types are:
- complete_request / complete_reply - for autocompletion
- inspect_request / inspect_reply - for signature/documentation help

**Patch do_complete on the Kernel**

The kernel's do_complete method handles complete_request messages. You can patch it:

```python
from ipykernel.ipkernel import IPythonKernel
from functools import wraps

# Store original
_original_do_complete = IPythonKernel.do_complete

@wraps(_original_do_complete)
def custom_do_complete(self, code, cursor_pos):
    """Enhanced completion with ranking."""
    # Get original reply
    reply = _original_do_complete(self, code, cursor_pos)
    
    if reply['status'] == 'ok':
        matches = reply['matches']
        
        # Custom ranking logic
        def rank(name):
            ...
        
        # Filter __dunder__ unless explicitly typing them
        text = code[:cursor_pos].split()[-1] if code[:cursor_pos].split() else ''
        if '__' not in text:
            matches = [m for m in matches if not m.startswith('__')]
        
        # Sort by rank
        reply['matches'] = sorted(matches, key=rank)
    
    return reply

# Apply patch
IPythonKernel.do_complete = custom_do_complete
```

**Patch do_inspect for Signature Help**

For enhanced signature/documentation help:

```python
from ipykernel.ipkernel import IPythonKernel
from jedi import Interpreter
from functools import wraps

_original_do_inspect = IPythonKernel.do_inspect

@wraps(_original_do_inspect)
def custom_do_inspect(self, code, cursor_pos, detail_level=0):
    """Enhanced inspect with Jedi signatures."""
    # Try Jedi first for better signature info
    try:
        ns = self.shell.user_ns
        
        # Get signatures at cursor
        ...
            
            return {
                'status': 'ok',
                'found': True,
                'data': {'text/markdown': text, 'text/plain': sig.docstring()},
                'metadata': {}
            }
    except Exception:
        pass
    
    # Fall back to original
    return _original_do_inspect(self, code, cursor_pos, detail_level)

IPythonKernel.do_inspect = custom_do_inspect
```

**Create an IPython Extension**

Package everything as a proper IPython extension:

```python
def load_ipython_extension(ip):
    """Called when extension is loaded via %load_ext"""
    from ipykernel.ipkernel import IPythonKernel
    from functools import wraps
    
    # Patch do_complete
    _orig = IPythonKernel.do_complete
    
    @wraps(_orig)
    def patched_do_complete(self, code, cursor_pos):
        ...
    
    IPythonKernel.do_complete = patched_do_complete
    print("Enhanced completions loaded!")

def unload_ipython_extension(ip):
    """Called when extension is unloaded"""
    # Restore original if needed
    pass
```

Usage:

```
%load_ext my_completer_extension
```

Or add to ipython_config.py:

```
c.InteractiveShellApp.extensions = ['my_completer_extension']
```

In [ ]:
what do xpush(), _get_info() and publish() do exactly ?

#### Send commands and display data to the Javascript frontend

A convenience wrapper that sends a command to the frontend via the transient mechanism.

```python
InteractiveShell.run_cmd(cmd, data='', meta=None, update=False, **kw)
    InteractiveShell.transient(data='', subtype='plain', mimetype='text', meta=None, update=False, **kw)
```

In Jupyter, when you output something, it gets sent to the frontend via a display_data or execute_result message. These messages have three main parts:

```json
{
    "data": {"text/plain": "Hello", "text/html": "<b>Hello</b>"},  # The content
    "metadata": {},  # Extra info about the content
    "transient": {}  # Data that should NOT be persisted in the notebook
}
```

Sends display data to the frontend where the actual payload is in the transient field, not the main data field.

The key insight: transient data is displayed but not saved when the notebook is saved. It's ephemeral.

Why use transient?
- Not saved to notebook - Commands and temporary UI updates don't clutter the saved .ipynb file
- Custom frontend communication - The solveit frontend watches for specific transient keys
- Ephemeral state - Progress indicators, status updates, commands that shouldn't persist

Example usage:

```python
run_cmd("scroll_to", msg_id="abc123")
```

### Web access

I have access to two main tools:

1. Web Search - I can search for current information, recent events, technical documentation, news, and anything where up-to-date data would be helpful. I use this when:
- You need recent information (after my March 2025 knowledge cutoff)
- You're looking for specific facts or current conditions
- Real-time data is important (weather, news, stock prices, etc.)

2. Read URL - I can fetch and read content from specific web pages you provide, which is useful for analyzing articles, documentation, or other online content.

I use these tools strategically - I don't search for things I already know well (like general programming concepts, historical facts, or established knowledge), but I will search when current or specific information would genuinely help answer your question.

#### Model inference service

The web_search function isn't defined in your Python environment - it's a tool that's available to me (the AI assistant) but not directly accessible to you in your Python code.

When I use web_search, I'm calling it through my own tool-calling mechanism, not through your Python interpreter. It's part of my capabilities, separate from the Python environment you're working in.

```python
web_search(query: str)
```

Parameters
- Takes a search query string and returns web search results

#### ipykernel_helper

Full-featured URL reader that can extract sections, convert to markdown, handle math (KaTeX/MathJax), absolutify image URLs, and optionally tag images for AI processing

```python
read_url(
    url: str,
    as_md: bool = True,
    extract_section: bool = True,
    selector: str = None
)
```

Parameters
- url: The URL to read (required)
- as_md: Whether to convert HTML to Markdown (default: True)
- extract_section: If URL has an anchor, return only that section (default: True)
- selector: CSS selector to extract specific sections using BeautifulSoup.select syntax (optional)

Uses internally
- scrape_url() - Fetch URL content using cloudscraper (handles anti-bot protections)
- get_md() - Convert HTML to clean markdown


### Code execution

In [ ]:
#| export
# import ...